In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
# Define paths to the dataset
dataset_dir = "/content/drive/MyDrive/gender_dataset"

# Define parameters
IMG_SIZE = 224
BATCH_SIZE = 32

# Create data generators with data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # Use 20% of the data for validation
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',  # binary labels for two classes
    subset='training',
)

validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
)


Found 1337 images belonging to 2 classes.
Found 334 images belonging to 2 classes.


In [ ]:
# Load the pre-trained ResNet50 model, excluding the top layers
base_model = ResNet50(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                      include_top=False,
                      weights='imagenet')

# Freeze most of the base model's layers, leaving the last few layers trainable
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Add custom layers on top of the base model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),  # L2 regularization
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model with a low learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print the model summary
# model.summary()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Define the number of epochs
EPOCHS = 20

# Callbacks for learning rate reduction and early stopping
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[lr_schedule, early_stopping]
)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


41/41 ━━━━━━━━━━━━━━━━━━━━ 394s 8s/step - accuracy: 0.4699 - loss: 1.2028 - val_accuracy: 0.5031 - val_loss: 1.1704 - learning_rate: 1.0000e-05
Epoch 2/20
 1/41 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 0.4688 - loss: 1.2506

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


41/41 ━━━━━━━━━━━━━━━━━━━━ 6s 140ms/step - accuracy: 0.4688 - loss: 1.2506 - val_accuracy: 0.4286 - val_loss: 1.1981 - learning_rate: 1.0000e-05
Epoch 3/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 146s 2s/step - accuracy: 0.5269 - loss: 1.1634 - val_accuracy: 0.4906 - val_loss: 1.1635 - learning_rate: 1.0000e-05
Epoch 4/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5938 - loss: 1.1206 - val_accuracy: 0.7143 - val_loss: 1.0793 - learning_rate: 1.0000e-05
Epoch 5/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 95s 2s/step - accuracy: 0.5674 - loss: 1.1262 - val_accuracy: 0.5063 - val_loss: 1.1460 - learning_rate: 1.0000e-05
Epoch 6/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5938 - loss: 1.0923 - val_accuracy: 0.3571 - val_loss: 1.1946 - learning_rate: 1.0000e-05
Epoch 7/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - accuracy: 0.5743 - loss: 1.1238 - val_accuracy: 0.5063 - val_loss: 1.1523 - learning_rate: 1.0000e-05
Epoch 8/20
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5312 - loss: 1.1074 

In [ ]:
# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")


In [ ]:
# Load and preprocess an image for prediction
from tensorflow.keras.preprocessing import image

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize

    # Predict the class
    prediction = model.predict(img_array)
    if prediction < 0.5:
        print("Prediction: Man")
    else:
        print("Prediction: Woman")

# Test with a sample image
predict_image("/content/maa.jpg")